code_chunk = new LP model, code_chunk_2 = old LP model

In [16]:
code_chunk = """Here is the modified Python code that incorporates new strategies to address the issues identified in the performance feedback:
```python
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the function
def optimize_production():
    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4
    ressources_total = 4

    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
    ]

    production_time = [
        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
    ]

    initial_inventory = [41.0, 58.0, 31.0, 38.0]
    yield_loss = [
        [0.9, 0.7, 0.9, 0.8],
        [0.8, 0.6, 0.95, 0.8],
        [0.9, 0.75, 0.9, 0.78],
        [0.95, 0.87, 0.97, 0.98],
    ]
    capacity_reduction_factor = 0.9

    worker_hours_per_product = 2.0
    hourly_wage = 20.0
    hourly_wage_overtime = 35.0
    hiring_cost = 10.0
    layoff_cost = 5.0
    initial_workers = 10

    # Create the problem
    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)

    # Index ranges for products, workstations, and periods
    products = range(products_total)
    workstations = range(workstations_total)
    periods = range(time_horizon)

    # Decision Variables
    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]
    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]
    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]
    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]

    # New strategy for production: adjust production ratios to meet demand better and avoid missed sales
    for i in products:
        for t in periods:
            problem += Produced[i][t] >= min_sales[i][t] * 0.8, f"Min_Production_{i}_{t}"
            problem += Produced[i][t] <= max_demand[i][t] * 1.2, f"Max_Production_{i}_{t}"

    # New strategy for slow-moving inventory: reduce production for products with low demand
    for i in products:
        if i == 1:  # product 1 has low demand
            for t in periods:
                problem += Produced[i][t] <= 0.5 * max_demand[i][t], f"Reduced_Production_{i}_{t}"

    # Objective Function
    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)
    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)
    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)
    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)
    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)
    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)
    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term

    # Constraints
    for i in products:
        for t in periods:
            # Sales constraints
            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"
            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"
            # Inventory balance constraint
            if t == 0:
                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"
            else:
                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"

    for j in workstations:
        for t in periods:
            # Adjusted capacity constraint
            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"

    for t in periods:
        # Workforce constraints
        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"
        if t == 0:
            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"
        else:
            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"

    # Solve the problem
    problem.solve()

    # Output the results
    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]

    # Extract the optimization results
    results = {
        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],
        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],
        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],
        "Workforce": [Workforce[t].varValue for t in periods],
        "Hires": [Hires[t].varValue for t in periods],
        "Firings": [Firings[t].varValue for t in periods],
        "Overtime": [Overtime[t].varValue for t in periods],
        "Profit": pulp.value(problem.objective)
    }

    for key, value in results.items():
        print(f"{key}: {value}")

    return results

# Table: New Strategies
# | Strategy | Description | Reason for Implementation |
# | --- | --- | --- |
# | Adjust production ratios | Adjust production ratios to meet demand better and avoid missed sales | Missed sales in periods with high demand |
# | Reduce production for slow-moving inventory | Reduce production for products with low demand to prevent excess inventory | Excess inventory for slow-moving products |

# Call the function to optimize production
optimize_production()
```
The new strategy added is to reduce production for products with low demand to prevent excess inventory. This is done by adding a constraint to reduce production for product 1, which has low demand.
"""

In [17]:
code_chunk_2 = """```python
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the function
def optimize_production():
    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4
    ressources_total = 4

    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
    ]

    production_time = [
        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
    ]

    initial_inventory = [41.0, 58.0, 31.0, 38.0]
    yield_loss = [
        [0.9, 0.7, 0.9, 0.8],
        [0.8, 0.6, 0.95, 0.8],
        [0.9, 0.75, 0.9, 0.78],
        [0.95, 0.87, 0.97, 0.98],
    ]
    capacity_reduction_factor = 0.9

    # New data required for the function
    worker_hours_per_product = 2.0  # Assumption
    hourly_wage = 20.0  # Assumption
    hourly_wage_overtime = 35.0  # Assumption
    hiring_cost = 10.0  # Assumption
    layoff_cost = 5.0  # Assumption
    initial_workers = 10  # Assumption

    # Create the problem
    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)

    # Index ranges for products, workstations, and periods
    products = range(products_total)
    workstations = range(workstations_total)
    periods = range(time_horizon)

    # Decision Variables
    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]
    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]
    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]
    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]

    # Objective Function
    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)
    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)
    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)
    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)
    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)
    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)
    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term

    # Constraints
    for i in products:
        for t in periods:
            # Sales constraints
            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"
            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"
            # Inventory balance constraint
            if t == 0:
                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"
            else:
                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"

    for j in workstations:
        for t in periods:
            # Adjusted capacity constraint
            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= \
                       capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"

    for t in periods:
        # Workforce constraints
        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"
        if t == 0:
            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"
        else:
            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"

    # Solve the problem
    problem.solve()

    # Output the results
    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]

    # Extract the optimization results
    results = {
        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],
        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],
        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],
        "Workforce": [Workforce[t].varValue for t in periods],
        "Hires": [Hires[t].varValue for t in periods],
        "Firings": [Firings[t].varValue for t in periods],
        "Overtime": [Overtime[t].varValue for t in periods],
        "Profit": pulp.value(problem.objective)
    }

    for key, value in results.items():
        print(f"{key}: {value}")

    return results

# Call the function to optimize production
optimize_production()
```"""


Input from the previous process:

In [18]:
def extract_python_code_manual(text):
    """
    Manually extracts Python code enclosed between ```python and ``` markers.

    Parameters:
    - text (str): The input text containing the Python code blocks.

    Returns:
    - list of str: A list containing all extracted Python code blocks.
    """
    code_blocks = []
    capture = False
    current_block = []

    # Split the text into lines for line-by-line processing
    lines = text.split('\n')

    for line in lines:
        # Check if the line contains the start marker
        if line.strip() == '```python':
            capture = True
            current_block = []  # Start a new block
        elif line.strip() == '```' and capture:
            # If we find the end marker and are in capture mode, stop capturing
            capture = False
            # Add the current block to the list of code blocks
            code_blocks.append('\n'.join(current_block))
        elif capture:
            # If we are in capture mode, add the line to the current block
            current_block.append(line)

    return code_blocks

In [19]:
!pip install simpy 


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import numpy as np
import simpy
import random

def run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, seed=42):
    random.seed(seed)
    np.random.seed(seed)

    num_periods = 6
    back_orders_penalty = [5, 6, 7, 2]
    initial_inventory = [41.0, 58.0, 31.0, 38.0]

    # Specific profits and holding costs for each product
    profit_per_unit = [132.0, 83.0, 225.0, 131.0]
    holding_cost_per_unit = [3.0, 8.0, 5.0, 1.0]

    worker_hours_per_product = 2.0  # Assumption
    hourly_wage = 20.0  # Assumption
    hourly_wage_overtime = 35.0  # Assumption
    hiring_cost = 10.0  # Assumption
    layoff_cost = 5.0  # Assumption
    initial_workers = 10  # Assumption

    production_time = [
        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
    ]

    yield_loss = [
        [0.9, 0.7, 0.9, 0.8],
        [0.8, 0.6, 0.95, 0.8],
        [0.9, 0.75, 0.9, 0.78],
        [0.95, 0.87, 0.97, 0.98],
    ]

    capacity_reduction_factor = 0.9
    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    Workstation_capacities = [25, 75, 30]

    product_workflows = [
        [0, 1],  # Product 0
        [1, 2],  # Product 1
        [0],     # Product 2
        [1, 2],  # Product 3
    ]

    env = simpy.Environment()

    class Workstation:
        def __init__(self, env, capacity, id):
            self.env = env
            self.capacity = capacity
            self.id = id
            self.queue = simpy.Resource(env, capacity=capacity)
            self.working = True

        def fail(self, duration):
            self.working = False
            yield self.env.timeout(duration)
            self.working = True

        def process(self, produced):
            if self.working:
                with self.queue.request() as request:
                    yield request
                    processing_time = produced / self.capacity
                    yield self.env.timeout(processing_time)
            else:
                yield self.env.timeout(1)  # Wait for 1 unit of time before trying again

    class WorkerShift:
        def __init__(self, env, shifts):
            self.env = env
            self.shifts = shifts
            self.current_shift = 0
            self.current_workers = shifts[0]['workers']

        def change_shift(self):
            while True:
                yield self.env.timeout(8)  # Change shift every 8 hours
                self.current_shift = (self.current_shift + 1) % len(self.shifts)
                self.current_workers = self.shifts[self.current_shift]['workers']
                print(f"Shift change at time {self.env.now:.2f}. Current workers: {self.current_workers}")

    class MultiProductFactory:
        def __init__(self, env, workstations, worker_shift):
            self.env = env
            self.inventory = list(initial_inventory)
            self.total_profits = 0.0
            self.period_results = []
            self.workstations = workstations
            self.worker_shift = worker_shift
            self.resources = [list(cap) for cap in capacity]  # Initialize resource capacities

        def process_at_workstation(self, workstation, produced, product_id):
            with workstation.queue.request() as request:
                yield request
                processing_time = produced / workstation.capacity
                yield self.env.timeout(processing_time)

        def run_factory(self):
            for period in range(num_periods):
                print(f"\n--- Period {period + 1} ---")
                period_data = {'Period': period, 'Products': []}
                total_produced = 0
                total_sales = 0

                # Simulate workstation outages randomly
                for ws in self.workstations:
                    if ws.id == 2:  # Example: Specific workstation id for potential outage
                        if np.random.poisson(1/5) >= 1:  # Poisson distribution to simulate outages
                            downtime_duration = 2  # Downtime duration in time units
                            print(f"Workstation {ws.id} starting downtime at time {self.env.now} for {downtime_duration} periods.")
                            yield self.env.process(ws.fail(downtime_duration))
                            print(f"Workstation {ws.id} ending downtime at time {self.env.now}.")

                for product_id in range(len(self.inventory)):
                    produced = produced_Prev[product_id][period]

                    # Resource requirements and checking capacities
                    for workstation_id in product_workflows[product_id]:
                        workstation = self.workstations[workstation_id]
                        required_resource = produced * production_time[workstation_id][period] / yield_loss[workstation_id][product_id]
                        if required_resource > self.resources[workstation_id][period]:
                            produced = self.resources[workstation_id][period] * yield_loss[workstation_id][product_id] / production_time[workstation_id][period]
                            self.resources[workstation_id][period] = 0
                        else:
                            self.resources[workstation_id][period] -= required_resource

                    self.inventory[product_id] += produced
                    total_produced += produced

                    if produced > 0:
                        yield self.env.process(self.process_at_workstation(workstation, produced, product_id))

                    fluctuation = random.uniform(-0.1, 0.1)
                    actual_demand = sale_Prev[product_id][period] * (1 + fluctuation)
                    actual_demand = min(actual_demand, max_demand[product_id][period])
                    sold = max(min(self.inventory[product_id], actual_demand), min_sales[product_id][period])
                    missed_sales = max(actual_demand - sold, 0)

                    revenue = sold * profit_per_unit[product_id]
                    holding_costs = self.inventory[product_id] * holding_cost_per_unit[product_id]
                    period_profit = revenue - holding_costs
                    self.total_profits += period_profit

                    self.inventory[product_id] = max(self.inventory[product_id] - sold, 0)

                    product_data = {
                        'Product ID': product_id,
                        'Produced': produced,
                        'Sold': sold,
                        'Missed Sales': missed_sales,
                        'Ending Inventory': self.inventory[product_id],
                        'Period Profit': period_profit
                    }
                    period_data['Products'].append(product_data)

                total_required_worker_hours = total_produced * worker_hours_per_product
                overtime_hours = max(total_required_worker_hours - self.worker_shift.current_workers * 8, 0)
                regular_hours = min(total_required_worker_hours, self.worker_shift.current_workers * 8)
                hiring_needed = max((total_required_worker_hours - regular_hours - overtime_hours) / 8, 0)
                layoffs_needed = max((self.worker_shift.current_workers * 8 - total_required_worker_hours) / 8, 0)

                self.worker_shift.current_workers += hiring_needed - layoffs_needed

                # Calculate workforce utilization
                available_worker_hours = self.worker_shift.current_workers * 8  # Assuming each worker works for 8 hours per shift
                workforce_utilization = total_required_worker_hours / available_worker_hours if available_worker_hours > 0 else 0

                overtime_cost = overtime_hours * hourly_wage_overtime
                regular_cost = regular_hours * hourly_wage
                hiring_cost_total = hiring_needed * hiring_cost
                layoff_cost_total = layoffs_needed * layoff_cost

                period_profit -= (overtime_cost + regular_cost + hiring_cost_total + layoff_cost_total)
                self.total_profits += period_profit

                period_data.update({
                    'Overtime Hours': overtime_hours,
                    'Regular Hours': regular_hours,
                    'Hired': hiring_needed,
                    'Fired': layoffs_needed,
                    'Workforce': self.worker_shift.current_workers,
                    'Workforce Utilization': workforce_utilization
                })

                self.period_results.append(period_data)
                yield self.env.timeout(1)

    produced_Prev = np.array(produced_Prev)
    sale_Prev = np.array(sale_Prev)
    min_sales = np.array(min_sales)

    workstations = [Workstation(env, capacity, id) for id, capacity in enumerate(Workstation_capacities)]
    worker_shift = WorkerShift(env, shifts=[
        {'start': 0, 'end': 8, 'workers': 223},
        {'start': 8, 'end': 16, 'workers': 230},
        {'start': 16, 'end': 24, 'workers': 55}
    ])
    factory = MultiProductFactory(env, workstations, worker_shift)

    env.process(worker_shift.change_shift())
    env.process(factory.run_factory())
    env.run(until=num_periods * 24)  # Ensure the environment runs through all periods

    return f"\nTotal Profits across all products and periods: {factory.total_profits:.2f}"


In [21]:
!pip install openai==0.28

import os
import openai

openai.api_key = ''
os.environ['OPENAI_API_KEY'] = openai.api_key


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
# Construct the chat history with the current data and previous solution
chat_history = [
{"role": "system", "content": "You are a helpful assistant."},
{"role": "user", "content": f"Consider the two codes: {code_chunk} and {code_chunk_2}. The first code contains some additional variables/parameters that are not inside the second one, like the 10 here in problem += Stock[i][t] >= 10... Give it a real parameter names and add the paramter values to the other values. Define the LP model as a function that uses these additional paramters (e.g. the xxx paratemter/factor=10 then in the arguments in the example I've shown you) as arguments. These numeircal parawmters present in the first code, yet not present in the second code must be defined seperatedly. There must be no other arguments given to the function but those additional parameters. Return 6 decision variables: produced, sales, overtime, workforce, hires, firings. E.g. arguments like: {x}. But beware: `sales_discount_high_demand` & `sales_discount_low_demand`: (reserved for future use, not actually implemented in this example, but you can use it in sale price adjustments for demand fluctuations). Is not acceptable. If a paramter is not used in the simulation it can also not be defined as argument"}
]

response = openai.ChatCompletion.create(model="gpt-4o", messages=chat_history)
print(response.choices[0].message.content)

# Save the alternative solution in the secondary_solutions list
expanded = response.choices[0].message.content

# At this point, secondary_solutions contains the new solutions corresponding to each item in data_list

Sure, let's refactor the provided code to include the additional parameters from the first snippet as arguments. These parameters will be incorporated into the LP model function, and we will use realistic parameter names. The function will then return six decision variables.

Here's the updated code for defining and running the LP model:

```python
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

def run_opt_model(min_production_ratio=0.8, max_production_ratio=1.2, low_demand_production_ratio=0.5,
                  capacity_reduction_factor=0.9, worker_hours_per_product=2.0, 
                  hourly_wage=20.0, hourly_wage_overtime=35.0, hiring_cost=10.0, 
                  layoff_cost=5.0, initial_workers=10):

    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4
    
    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 2

Extended LP

In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

def run_opt_model(min_production_ratio=0.8, max_production_ratio=1.2, low_demand_production_ratio=0.5):


    capacity_reduction_factor=0.9, 
    worker_hours_per_product=2.0, 
    hourly_wage=20.0, 
    hourly_wage_overtime=35.0, 
    hiring_cost=10.0, 
    layoff_cost=5.0, 
    initial_workers=10

    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4
    
    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
    ]

    production_time = [
        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
    ]

    initial_inventory = [41.0, 58.0, 31.0, 38.0]
    yield_loss = [
        [0.9, 0.7, 0.9, 0.8],
        [0.8, 0.6, 0.95, 0.8],
        [0.9, 0.75, 0.9, 0.78],
        [0.95, 0.87, 0.97, 0.98],
    ]

    # Create the problem
    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)

    # Index ranges for products, workstations, and periods
    products = range(products_total)
    workstations = range(workstations_total)
    periods = range(time_horizon)

    # Decision Variables
    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]
    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]
    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]
    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]

    # Constraints
    for i in products:
        for t in periods:
            # New strategy for production: adjust production ratios to meet demand better and avoid missed sales
            problem += Produced[i][t] >= min_sales[i][t] * min_production_ratio, f"Min_Production_{i}_{t}"
            problem += Produced[i][t] <= max_demand[i][t] * max_production_ratio, f"Max_Production_{i}_{t}"

        # New strategy for slow-moving inventory: reduce production for products with low demand
        if i == 1:  # product 1 has low demand
            for t in periods:
                problem += Produced[i][t] <= low_demand_production_ratio * max_demand[i][t], f"Reduced_Production_{i}_{t}"

    # Objective Function
    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)
    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)
    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)
    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)
    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)
    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)
    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term

    # Constraints
    for i in products:
        for t in periods:
            # Sales constraints
            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"
            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"
            # Inventory balance constraint
            if t == 0:
                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"
            else:
                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"

    for j in workstations:
        for t in periods:
            # Adjusted capacity constraint
            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"

    for t in periods:
        # Workforce constraints
        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"
        if t == 0:
            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"
        else:
            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"

    # Solve the problem
    problem.solve()

    # Extract the optimization results
    results = {
        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],
        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],
        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],
        "Workforce": [Workforce[t].varValue for t in periods],
        "Hires": [Hires[t].varValue for t in periods],
        "Firings": [Firings[t].varValue for t in periods],
        "Overtime": [Overtime[t].varValue for t in periods],
        "Profit": value(problem.objective)
    }

    for key, value in results.items():
        print(f"{key}: {value}")

    return results

# Example of calling the function with specific parameters
run_opt_model(min_production_ratio=0.8, max_production_ratio=1.2, low_demand_production_ratio=0.5)

In [7]:
x = """def run_opt_model(storage_capacity_factor=0.1, production_priority_high=0.5, production_priority_low=0.3,
                        sales_discount_high_demand=0.9, sales_discount_low_demand=0.8, peak_workforce_factor=1.2,
                        non_peak_workforce_factor=1.0):"""

In [8]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

def run_opt_model(storage_capacity_factor=0.1, 
                  production_priority_high=0.5, 
                  production_priority_low=0.3,
                  sales_discount_high_demand=0.9, 
                  sales_discount_low_demand=0.8, 
                  peak_workforce_factor=1.2,
                  non_peak_workforce_factor=1.0,
                  high_profit_products = 200,
                  low_sales = 50):
    
    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4
    ressources_total = 4

    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
    ]

    production_time = [
        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
    ]

    initial_inventory = [41.0, 58.0, 31.0, 38.0]
    yield_loss = [
        [0.9, 0.7, 0.9, 0.8],
        [0.8, 0.6, 0.95, 0.8],
        [0.9, 0.75, 0.9, 0.78],
        [0.95, 0.87, 0.97, 0.98],
    ]
    capacity_reduction_factor = 0.9

    # New data required for the function
    worker_hours_per_product = 2.0  # Assumption
    hourly_wage = 20.0  # Assumption
    hourly_wage_overtime = 35.0  # Assumption
    hiring_cost = 10.0  # Assumption
    layoff_cost = 5.0  # Assumption
    initial_workers = 10  # Assumption

    # Create the problem
    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)

    # Index ranges for products, workstations, and periods
    products = range(products_total)
    workstations = range(workstations_total)
    periods = range(time_horizon)

    # Decision Variables
    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]
    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]
    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]
    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]
    Storage_Capacity = [[LpVariable(f"Storage_Capacity_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]

    # Objective Function
    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)
    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)
    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)
    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)
    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)
    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)
    storage_cost_term = lpSum(storage_capacity_factor * Storage_Capacity[i][t] for i in products for t in periods)
    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term - storage_cost_term

    # Constraints
    for i in products:
        for t in periods:
            # Sales constraints
            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"
            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"
            # Inventory balance constraint
            if t == 0:
                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"
            else:
                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"
            # New constraint for storage capacity
            problem += Storage_Capacity[i][t] >= Stock[i][t], f"Storage_Capacity_{i}_{t}"

    for j in workstations:
        for t in periods:
            # Adjusted capacity constraint
            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"

    for t in periods:
        # Workforce constraints
        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"
        if t == 0:
            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"
        else:
            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"

    # Strategies
    for i in products:
        for t in periods:
            if profit[i] > high_profit_products:
                problem += Produced[i][t] >= production_priority_high * max_demand[i][t], f"Production_Priority_High_{i}_{t}"
            else:
                problem += Produced[i][t] >= production_priority_low * max_demand[i][t], f"Production_Priority_Low_{i}_{t}"

            if min_sales[i][t] < low_sales:
                problem += Sale[i][t] >= sales_discount_low_demand * min_sales[i][t], f"Sales_Discount_Low_{i}_{t}"
            else:
                problem += Sale[i][t] >= sales_discount_high_demand * min_sales[i][t], f"Sales_Discount_High_{i}_{t}"

    for t in periods:
        if t > 2:
            problem += Workforce[t] >= peak_workforce_factor * initial_workers, f"Workforce_Peak_{t}"
        else:
            problem += Workforce[t] >= non_peak_workforce_factor * initial_workers, f"Workforce_NonPeak_{t}"

    # Solve the problem
    problem.solve()

    # Extract and print the optimization results
    produced = [[Produced[i][t].varValue for t in periods] for i in products]
    sales = [[Sale[i][t].varValue for t in periods] for i in products]
    stock = [[Stock[i][t].varValue for t in periods] for i in products]
    workforce = [Workforce[t].varValue for t in periods]
    hires = [Hires[t].varValue for t in periods]
    firings = [Firings[t].varValue for t in periods]
    overtime = [Overtime[t].varValue for t in periods]

    results = {
        "Produced": produced,
        "Sale": sales,
        "Stock": stock,
        "Workforce": workforce,
        "Hires": hires,
        "Firings": firings,
        "Overtime": overtime
    }

    for key, value in results.items():
        print(f"{key}: {value}")

    return produced, sales, overtime, workforce, hires, firings

# Call the function to optimize production
run_opt_model()


Produced: [[93.316667, 68.4, 33.3, 143.4, -34.3, 76.883333], [24.615277, 0.0, 190.5, 105.5, 77.5, 45.0], [0.0, 97.0, 108.5, 237.5, 47.5, 71.5], [0.31097222, 17.689028, 73.880972, 77.7, 0.0, 60.9]]
Sale: [[71.0, 28.0, 110.0, 78.0, 45.0, 90.0], [61.615277, 21.0, 85.0, 211.0, 77.5, 45.0], [23.0, 105.0, 108.5, 190.0, 95.0, 71.5], [20.0, 36.0, 73.880972, 47.7, 30.0, 60.9]]
Stock: [[63.316667, 103.71667, 27.016667, 92.416667, 13.116667, 0.0], [21.0, 0.0, 105.5, 0.0, 0.0, 0.0], [8.0, 0.0, 0.0, 47.5, 0.0, 0.0], [18.310972, 0.0, 0.0, 30.0, 0.0, 0.0]]
Workforce: [236.48583, 366.17806, 812.36194, 812.36194, 181.4, 508.56667]
Hires: [226.48583, 129.69222, 446.18389, 0.0, 0.0, 327.16667]
Firings: [0.0, 0.0, 0.0, 0.0, 630.96194, 0.0]
Overtime: [0.0, 0.0, 0.0, 315.83806, 0.0, 0.0]


([[93.316667, 68.4, 33.3, 143.4, -34.3, 76.883333],
  [24.615277, 0.0, 190.5, 105.5, 77.5, 45.0],
  [0.0, 97.0, 108.5, 237.5, 47.5, 71.5],
  [0.31097222, 17.689028, 73.880972, 77.7, 0.0, 60.9]],
 [[71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
  [61.615277, 21.0, 85.0, 211.0, 77.5, 45.0],
  [23.0, 105.0, 108.5, 190.0, 95.0, 71.5],
  [20.0, 36.0, 73.880972, 47.7, 30.0, 60.9]],
 [0.0, 0.0, 0.0, 315.83806, 0.0, 0.0],
 [236.48583, 366.17806, 812.36194, 812.36194, 181.4, 508.56667],
 [226.48583, 129.69222, 446.18389, 0.0, 0.0, 327.16667],
 [0.0, 0.0, 0.0, 0.0, 630.96194, 0.0])

In [3]:
!pip install hyperopt


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import re
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK, STATUS_FAIL

# Function to extract profit from simulation output
def extract_profit_from_output(sim_output):
    match = re.search(r"Total Profits across all products and periods: (\d+\.\d+)", sim_output)
    if match:
        return float(match.group(1))
    else:
        raise ValueError("Profit not found in simulation output")



# Define the objective function to maximize profit
def objective(params):
    try:
        produced, sales, overtime, workforce, hires, firings = run_opt_model(
            storage_capacity_factor=params['storage_capacity_factor'],
            production_priority_high=params['production_priority_high'],
            production_priority_low=params['production_priority_low'],
            sales_discount_high_demand=params['sales_discount_high_demand'],
            sales_discount_low_demand=params['sales_discount_low_demand'],
            peak_workforce_factor=params['peak_workforce_factor'],
            non_peak_workforce_factor=params['non_peak_workforce_factor'],
        )

        sim_output = run_extended_simulation(
            produced, sales, overtime, workforce, hires, firings
        )

        total_profit = extract_profit_from_output(sim_output)
        return {'loss': -total_profit, 'status': STATUS_OK}  # Negative profit to minimize
    except Exception as e:
        print(f"Error during objective function evaluation: {e}")
        return {'loss': float('inf'), 'status': STATUS_FAIL}

# Define the search space
space = {
    'storage_capacity_factor': hp.uniform('storage_capacity_factor', 0, 1),
    'production_priority_high': hp.uniform('production_priority_high', 0, 1),
    'production_priority_low': hp.uniform('production_priority_low', 0, 1),
    'sales_discount_high_demand': hp.uniform('sales_discount_high_demand', 0, 1),
    'sales_discount_low_demand': hp.uniform('sales_discount_low_demand', 0, 1),
    'peak_workforce_factor': hp.uniform('peak_workforce_factor', 0, 5),
    'non_peak_workforce_factor': hp.uniform('non_peak_workforce_factor', 0, 5)
}

# Run the optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=1000, trials=trials)

print("Best Parameters found: ", best)

# Use the best parameters to run the optimization model
optimized_produced, optimized_sales, optimized_overtime, optimized_workforce, optimized_hires, optimized_firings = run_opt_model(
    storage_capacity_factor=best['storage_capacity_factor'],
    production_priority_high=best['production_priority_high'],
    production_priority_low=best['production_priority_low'],
    sales_discount_high_demand=best['sales_discount_high_demand'],
    sales_discount_low_demand=best['sales_discount_low_demand'],
    peak_workforce_factor=best['peak_workforce_factor'],
    non_peak_workforce_factor=best['non_peak_workforce_factor']
)

# Run the simulation model with optimized parameters
final_sim_output = run_extended_simulation(
    optimized_produced,
    optimized_sales,
    optimized_overtime,
    optimized_workforce,
    optimized_hires,
    optimized_firings
)

# Extract and print the final profit
final_profit = extract_profit_from_output(final_sim_output)
print(f"Final Optimized Profit: {final_profit}")


In [10]:
import numpy as np
import re
import hyperopt
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

# Function to extract profit from simulation output
def extract_profit_from_output(sim_output):
    match = re.search(r"Total Profits across all products and periods: (\d+\.\d+)", sim_output)
    if match:
        return float(match.group(1))
    else:
        raise ValueError("Profit not found in simulation output")

# Define the objective function to maximize profit
def objective(params):
    produced, sales, overtime, workforce, hires, firings = run_opt_model(
        storage_capacity_factor=params['storage_capacity_factor'],
        production_priority_high=params['production_priority_high'],
        production_priority_low=params['production_priority_low'],
        sales_discount_high_demand=params['sales_discount_high_demand'],
        sales_discount_low_demand=params['sales_discount_low_demand'],
        peak_workforce_factor=params['peak_workforce_factor'],
        non_peak_workforce_factor=params['non_peak_workforce_factor'],
        high_profit_products=params['high_profit_products'],
        low_sales=params['low_sales']
    )

    sim_output = run_extended_simulation(
        produced, sales, overtime, workforce, hires, firings
    )

    total_profit = extract_profit_from_output(sim_output)
    return {'loss': -total_profit, 'status': STATUS_OK}  # Negative profit to minimize

# Define the search space
space = {
    'storage_capacity_factor': hp.uniform('storage_capacity_factor', 0, 1),
    'production_priority_high': hp.uniform('production_priority_high', 0, 1),
    'production_priority_low': hp.uniform('production_priority_low', 0, 1),
    'sales_discount_high_demand': hp.uniform('sales_discount_high_demand', 0, 1),
    'sales_discount_low_demand': hp.uniform('sales_discount_low_demand', 0, 1),
    'peak_workforce_factor': hp.uniform('peak_workforce_factor', 0, 5),
    'non_peak_workforce_factor': hp.uniform('non_peak_workforce_factor', 0, 5),
    'high_profit_products': hp.uniform('high_profit_products', 0, 1000),
    'low_sales': hp.uniform('low_sales', 0, 1000)
}

# Run the optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=1000, trials=trials)

print("Best Parameters found: ", best)

# Use the best parameters to run the optimization model
optimized_produced, optimized_sales, optimized_overtime, optimized_workforce, optimized_hires, optimized_firings = run_opt_model(
    storage_capacity_factor=best['storage_capacity_factor'],
    production_priority_high=best['production_priority_high'],
    production_priority_low=best['production_priority_low'],
    sales_discount_high_demand=best['sales_discount_high_demand'],
    sales_discount_low_demand=best['sales_discount_low_demand'],
    peak_workforce_factor=best['peak_workforce_factor'],
    non_peak_workforce_factor=best['non_peak_workforce_factor'],
    high_profit_products=best['high_profit_products'],
    low_sales=best['low_sales']
)

# Run the simulation model with optimized parameters
final_sim_output = run_extended_simulation(
    optimized_produced,
    optimized_sales,
    optimized_overtime,
    optimized_workforce,
    optimized_hires,
    optimized_firings
)

# Extract and print the final profit
final_profit = extract_profit_from_output(final_sim_output)
print(f"Final Optimized Profit: {final_profit}")

Produced: [[30.0, 145.58341, 70.876133, 44.54046, 0.0, 121.31951], [0.0, 0.0, 243.27754, 134.7285, 47.761491, 57.467135], [5.7667038, 91.233296, 138.55965, 303.29877, 11.173137, 91.308892], [33.113804, 0.0, -65.222739, 164.53087, 0.0, 35.648118]]
Sale: [[71.0, 28.0, 110.0, 78.0, 45.0, 121.31951], [37.0, 21.0, 159.76753, 211.0, 55.0, 57.467135], [23.0, 105.0, 138.55965, 219.4719, 95.0, 91.308892], [20.0, 36.0, 9.0, 75.421939, 30.0, 35.648118]]
Stock: [[0.0, 117.58341, 78.45954, 45.0, 0.0, 0.0], [21.0, 0.0, 83.510005, 7.2385093, 0.0, 0.0], [13.766704, 0.0, 0.0, 83.826863, 0.0, 0.0], [51.113804, 15.113804, -59.108935, 30.0, 0.0, 0.0]]
Workforce: [137.76102, 473.63341, 774.98115, 774.98115, 117.86925, 611.4873]
Hires: [127.76102, 335.87239, 301.34775, 0.0, 0.0, 493.61805]
Firings: [0.0, 0.0, 0.0, 0.0, 657.1119, 0.0]            
Overtime: [0.0, 0.0, 0.0, 519.21605, 0.0, 0.0]          
                                                        
--- Period 1 ---
                                 

job exception: too many values to unpack (expected 2)



  0%|          | 0/1000 [00:00<?, ?trial/s, best loss=?]


ValueError: too many values to unpack (expected 2)

In [ ]:
def run_opt_model(storage_capacity_factor=0.1, production_priority_high=0.5, production_priority_low=0.3,
                        sales_discount_high_demand=0.9, sales_discount_low_demand=0.8, peak_workforce_factor=1.2,
                        non_peak_workforce_factor=1.0):

In [63]:
final_profit

790494.13

In [17]:
with open("optimization_results.txt", "w") as f:
    f.write(f"Best Parameters: {best}\n")
    f.write(f"Final Optimized Profit: {final_profit}\n")

New Use Case